# The Spinning Effective One-Body Hamiltonian

### Author: Tyler Knowles

## This module documents the reduced spinning effective one-body Hamiltonian as numerically implemented in LALSuite's SEOBNRv3 gravitational waveform approximant.

### Throughout this module, we will refer to [Barausse and Buonanno (2010)](https://arxiv.org/abs/0912.3517) as BB2010 and [Barausse and Buonanno (2011)](https://arxiv.org/abs/1107.2904) as BB2011.

<font color='green'>**This module has been validated against the LALSuite SEOBNRv3/SEOBNRv3_opt code that was reviewed and approved for LIGO parameter estimation by the LIGO Scientific Collaboration.**</font>

<a id='toc'></a>

# Table of Contents
$$\label{toc}$$

This module is organized as follows:

0. [Introduction](#intro): The Physical System of Interest
1. [Step 1](#Hreal): The Real Hamiltonian $H_{\rm real}$
1. [Step 2](#Heff): The Effective Hamiltonian $H_{\rm eff}$
1. [Step 3](#Hs): Leading Order Spin Effects $H_{\rm S}$
1. [Step 4](#Hns): The Nonspinning Hamiltonian $H_{\rm NS}$
1. [Step 5](#Hd): The Quadrupole Deformation $H_{\rm D}$
1. [Step N](#metric): The Metric $g$ of Choice
1. [Notes](#notes)
1. [Output](#latex_pdf_output): Output this module to $\LaTeX$-formatted PDF

<a id='intro'></a>

# Introduction: The Physical System of Interest \[Back to [top](#toc)\]
$$\label{intro}$$

Consider two black holes with masses $m_{1}$, $m_{2}$ and spins ${\bf S}_{1}$, ${\bf S}_{2}$ in a binary system described in spherical coordinates $\left( r, \theta, \phi \right)$.  Denote by

\begin{equation}
    M = m_{1} + m_{2}
\end{equation}

the total mass of this binary system.  The spinning effective one-body ("SEOB") Hamiltonian $H_{\rm real}$ describes the dynamics of this binary system via transformation to an effective Hamiltonian $H_{\rm eff}$ describing the motion of a test particle of mass

\begin{equation}
    \mu = \frac{m_{1} m_{2}}{M}
\end{equation}

and spin ${\bf S}^{*}$ in a deformed Kerr metric of mass $M$ and spin ${\bf S}_{\rm Kerr}$.  Denote also

\begin{equation}
    \eta = \frac{ \mu }{ M }.
\end{equation}

We will use Greek letters to denote spacetime tensor indicies, and spacial tensor indicies with lowercase Latin letters.  Throughout we take $c = 1$.

<a id='Hreal'></a>

# Step 1: The real Hamiltonian $H_{\textrm{real}}$ \[Back to [top](#toc)\]
$$\label{Hreal}$$

The SEOB Hamiltonian $H_{\rm real}$ is given by Equation (5.69) of BB2010:

\begin{equation}
    H_{\rm real} = M \sqrt{ 1 + 2 \eta \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) }.
\end{equation}

<font color='red'>The code in LALSimIMRSpinEOBHamiltonianPrec.c claims the origin of $H_{\textrm{real}}$ is Equation (2) of [Taracchini, Pan, Buonanno, et al](https://arxiv.org/abs/1202.0790):

\begin{equation}
    H_{\rm real} = M \sqrt{ 1 + 2 \nu \left( \frac{ H_{\rm eff} }{ \mu } - 1 \right) } - M
\end{equation}

where $\nu = \frac{\mu}{M}$.  Double-check this!</font>

<a id='Heff'></a>

# Step 2: The Effective Hamiltonian $H_{\rm eff}$ \[Back to [top](#toc)\]
$$\label{Heff}$$

The effective Hamiltonian $H_{\rm eff}$ is given by Equation (5.70) of BB2010.  We combine the notation of BB2010 Equations (5.70) and (5.43) to write

\begin{equation}
    H_{\rm eff} = H_{\rm S} + H_{\rm NS} \underbrace{- \frac{ \mu }{ 2 M r^{3} } \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j}}_{\rm H_{\rm D}}.
\end{equation}

Here $H_{\rm S}$ denotes leading order effects of spin-spin and spin-orbit coupling, $H_{\rm NS}$ is the Hamiltonian for a nonspinning test particle, and $H_{\rm D}$ describes quadrupole deformation of the coupling of the particle's spin with itself to leading order.

<a id='Hs'></a>

# Step 3: Leading Order Spin Effects $H_{\rm S}$ \[Back to [top](#toc)\]
$$\label{Hs}$$

From BB2010 Equation (3.4),

\begin{equation}
    H_{\rm S} = H_{\rm SO} + H_{\rm SS}
\end{equation}

where $H_{\rm SO}$ includes spin-orbit terms and $H_{\rm SS}$ includes spin-spin terms.

## Spin-orbit term $H_{\rm SO}$

From BB2010 Equation 

## Spin-spin term $H_{\rm SS}$

From BB2010 Equation (4.19),

\begin{eqnarray}                                                                                                                                                                                            
H_{\rm SS} &=& \omega \left( {\bf S} \cdot {\bf \hat{S}}_{\rm Kerr} \right) + \frac{ e^{-3 \tilde{\mu} -\nu} \tilde{J} \omega_{r} }{ 2 \tilde{B} \left( \sqrt{Q} + 1 \right) \sqrt{Q} \xi^{2} } \left\{ -e^{\tilde{\mu} + \nu} \left( {\bf \hat{p}} \cdot {\bf v} r \right) \left( {\bf \hat{p}} \cdot {\bf \xi} r \right) \left( {\bf S} \cdot {\bf \xi} \right)
\tilde{B} + e^{2 \left( \tilde{\mu} + \nu \right)} \left( {\bf \hat{p}} \cdot {\bf \xi} r \right)^2 \left( {\bf S}
\cdot {\bf v} \right) \right. \nonumber \\
&& \left. + e^{2 \tilde{\mu}} \left( 1 + \sqrt{Q} \right) \sqrt{Q} \left( {\bf S} \cdot {\bf v} \right)\xi^2 \tilde{B}^{2} + \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left[ \left( {\bf \hat{p}} \cdot {\bf v} r \right)
\left( {\bf S} \cdot {\bf n}\right) - \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf S} \cdot {\bf v} \right)\right] \xi^{2} \tilde{B}^{2} \right\} \nonumber \\
&& + \frac{ e^{-3 \tilde{\mu} - \nu} \omega_{\cos\theta} }{ 2 \tilde{B} \left( \sqrt{Q} + 1 \right) \sqrt{Q} } \left\{ -e^{2 \left( \tilde{\mu} + \nu \right)} \left( {\bf \hat{p}} \cdot {\bf \xi} r \right)^{2} \left( {\bf S} \cdot {\bf n} \right) + e^{\tilde{\mu} +\nu} \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf \hat{p}} \cdot {\bf \xi} r \right) \left( {\bf S} \cdot {\bf \xi} \right) \tilde{B} \right. \nonumber \\
&& \left. + \left[ \left( {\bf S} \cdot {\bf n} \right) \left( {\bf \hat{p}} \cdot {\bf v} r \right)^{2} - \tilde{J} \left( {\bf \hat{p}} \cdot {\bf n} \right) \left( {\bf S} \cdot {\bf v} \right) \left( {\bf \hat{p}} \cdot {\bf v} r\right) - e^{2 \tilde{\mu}} \left( 1 + \sqrt{Q} \right) \sqrt{Q} \left( {\bf S} \cdot {\bf n} \right) \xi^{2} \right] \tilde{B}^{2} \right\}
\end{eqnarray}

where

\begin{align}
\tilde{B} &= BR = \sqrt{ \Delta } \\
\tilde{B}_r &= B_r R = \frac{ r - M - \sqrt{\Delta} }{ \sqrt{\Delta} } \\
e^{2 \tilde{\mu}} &= e^{2\mu} R^2 = \Sigma \\
\tilde{J} &= J R = \sqrt{\Delta} \\
\hat{\bf S}_{\rm Kerr} &= \frac{ {\bf S}_{\rm Kerr} }{ \left\lvert {\bf S}_{\rm Kerr} \right\rvert }
\end{align}

<a id='Hns'></a>

# Step 4: The Nonspinning Hamiltonian $H_{\rm NS}$ \[Back to [top](#toc)\]
$$\label{Hns}$$

From BB2010 Equation (5.43),

\begin{equation}
    H_{\rm NS} = \beta^{i} p_{i} + \alpha \sqrt{ \mu^{2} + \gamma^{ij} p_{i} p_{j} + {\cal Q}_{4} \left( p \right) }
\end{equation}

where $p_{i}$ are spatial components of momentum in EOB coordinates and ${\cal Q}_{4}$ is quartic in $p_{i}$.  BB2010 specifies, in Equations (5.44)--(5.46), that for a spacetime metric $g$

\begin{align}
    \alpha &= \frac{ 1 }{ \sqrt{ - g^{tt} }} \\
    \beta^{i} &= \frac{ g^{ti} }{ g^{tt} } \\
    \gamma^{ij} &= g^{ij} - \frac{ g^{ti} g ^{tj} }{ g^{tt} }.
\end{align}

<a id='Hd'></a>

# Step 5: The Quadrupole Deformation $H_{\rm D}$ \[Back to [top](#toc)\]
$$\label{Hd}$$

We break up $H_{\rm D}$ as follows:

\begin{equation}
    H_{\rm D} = \underbrace{ -\frac{ \mu }{ 2 M r^{3} } }_{H_{\rm D}\ {\rm Term\ 1}} \underbrace{ \left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j} }_{H_{\rm D}\ {\rm Term\ 2}}
\end{equation}

## $H_{\rm D}$ Term 1: $-\frac{ \mu }{ 2 M r^{3} }$

$H_{\rm D}$ Term 1 can be easily written entirely in terms of intrinsic parameters: $m_{1}$, $m_{2}$, and $r$.  That is, using definitions of $\mu$ and $M$ given in the [Introduction](#intro) we may write

\begin{equation}
    H_{\rm D}\ {\rm Term\ 1} = -\frac{ m_{1} m_{2} }{ 2 \left( m_{1} + m_{2} \right)^{2}  r^{3} }.
\end{equation}

## $H_{\rm D}$ Term 2: $\left( \delta^{ij} - 3 n^{i} n^{j} \right) S^{*}_{i} S^{*}_{j}$

As is common in physics, $\delta^{ij}$ is the Kronecker delta:

\begin{equation}
    \delta_{ij} = \left\{ \begin{array}{cc}
        0, & i \not= j \\
        1, & i = j. \end{array} \right.
\end{equation}

Here ${\bf n}$ denotes normalized coordinates; e.g. if ${\bf R}$ denotes the spherical spacial coordinate vector ${\bf R} = \left( r, \theta, \phi \right)$, then ${\bf n} = \frac{\bf R }{ R }$ where $R = r$ is the magnitude of ${\bf R}$.  

From BB2010 Equation (5.63),

\begin{equation}
    {\bf S}^{*} = \underbrace{ \boldsymbol{\sigma}^{*} }_{ H_{\rm D}\ {\rm Term\ 2a} } + \underbrace{ \frac{1}{c^{2}} \boldsymbol{\Delta}_{\sigma^{*}} }_{H_{\rm D}\ {\rm Term\ 2b}}.
\end{equation}

## $H_{\rm D}$ Term 2a: $\boldsymbol{\sigma}^{*}$

From BB2010 Equation (5.3) we have

\begin{equation}
    \boldsymbol{\sigma}^{*} = \frac{ m_{1} }{ m_{2} } \left( {\bf S}_{1} + {\bf S}_{2} \right),
\end{equation}

which is in terms of the intrinsic physical parameters.

## $H_{\rm D}$ Term 2b: $\frac{1}{c^{2}} \boldsymbol{\Delta}_{\sigma^{*}}$

From BB2010 Equation (5.68) we have

\begin{equation}
    \boldsymbol{\Delta}_{\sigma^{*}} = \frac{ \eta }{ 12 } \left[ \frac{ 2M }{ r } \left( 7 \boldsymbol{\sigma}^{*} - 4 \boldsymbol{\sigma} \right) + \left( Q - 1 \right) \left( 3 \boldsymbol{\sigma} + 4 \boldsymbol{\sigma}^{*} \right) - 6 \frac{ \Delta_r }{ \Sigma } \left( \hat{\bf p} \cdot {\bf n} \right)^{2} \left( 6 \boldsymbol{\sigma} + 5 \boldsymbol{\sigma}^{*} \right) \right]
\end{equation}

where the hatted coordinates are scaled by $\mu$; e.g. $\hat{\bf p} = \frac{ {\bf p} }{ \mu }$.  From BB2010 Equation (5.2)

\begin{equation}
    \boldsymbol{\sigma} = {\bf S}_{1} + {\bf S}_{2};
\end{equation}

from BB2010 Equation (5.52)

\begin{equation}
    Q = 1 + \frac{ \Delta_r \left( \hat{\bf p} \cdot {\bf n} \right)^{2} }{ \Sigma } + \frac{ \left( \hat{\bf p} \cdot \boldsymbol{\xi} r \right)^{2} \Sigma }{ \Lambda_t \sin^{2} \theta } + \frac{ \left( \hat{\bf p} \cdot {\bf v} r \right)^{2} }{ \Sigma \sin^{2} \theta };
\end{equation}

from BB2010 Equation (5.38)

\begin{equation}
    \Delta_{r} = \Delta_{t} D^{-1}(u);
\end{equation}

from BB2010 Equation (4.5)

\begin{equation}
    \Sigma = r^{2} + a^{2} \cos^{2} \theta;
\end{equation}

from BB2010 Equation (3.23)

\begin{equation}
    \boldsymbol{\xi} = {\bf e}_{Z} \times {\bf N},
\end{equation}

where ${\bf e}_{Z}$ is a Cartesian unit vector in the $z$-direction and ${\bf N} = {\bf n}$ from BB2010 Equation (3.37); from BB2010 Equation (5.37)

\begin{equation}
    \Delta_{t} = r^{2} \left[ A(u) + \frac{ a^{2} }{ M^{2} } u^{2} \right];
\end{equation}

from BB2010 Equation (3.39)

\begin{equation}
    {\bf v} = {\bf n} \times \boldsymbol{\xi};
\end{equation}

from BB2010 Equation (5.42)

\begin{equation}
    D^{-1}(u) = 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3};
\end{equation}

from BB2010 discussion following Equation (5.40)

\begin{equation}
    u = \frac{ M }{ r };
\end{equation}

from BB2010 Equation (4.9)

\begin{equation}
    a = \frac{ \left\lvert {\bf S}_{\rm Kerr} \right\rvert }{ M };
\end{equation}

and from BB2010 Equation (5.41)

\begin{equation}
    A(u) = 1 - 2u + 2 \eta u^{3} + \eta \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) u^{4}.
\end{equation}

Oof.

<a id='metric'></a>

# Step N: The Metric $g$ of Choice \[Back to [top](#toc)\]
$$\label{metric}$$

In BB2010 Equations (5.36a)--(5.36e), we are given the appropriate choice of metric via

\begin{align}
    g^{tt} &= -\frac{ \Lambda_{t} }{ \Delta_{t} \Sigma } \\
    g^{rr} &= \frac{ \Delta_{r} }{ \Sigma } \\
    g^{\theta\theta} &= \frac{ 1 }{ \Sigma } \\
    g^{\phi\phi} &= \frac{ 1 }{ \Lambda_{t} } \left( -\frac{ \tilde{\omega}_{\rm fd}^{2} }{ \Delta_{t} \Sigma } + \frac{ \Sigma }{ \sin^{2} \theta } \right) \\
    g^{t\phi} &= -\frac{ \tilde{\omega}_{\rm fd} }{ \Delta_{t} \Sigma }
\end{align}

From BB2010 Equations (5.71), we have

\begin{equation}
    \Delta_{t} = r^{2} \Delta_{u}(u)
\end{equation}

and from Equations (5.73)--(5.76) we have

\begin{align}
\Delta_{u} &= \bar{\Delta}_u(u) \left[ 1 + \eta \Delta_0 + \eta \log \left( 1 + \Delta_1 u + \Delta_2 u^2 + \Delta_3 u^3 + \Delta_4 u^4 \right) \right] \\
    \bar{\Delta}_u(u) &= \frac{ a^{2} u^{2} }{ M^{2} } + \frac{ 2 u }{ \eta K - 1 } + \frac{ 1 }{ \left( \eta K - 1 \right)^{2} }
\end{align}

where $K$ is an adjustable parameter.  The coefficients $\Delta_{0}, \Delta_{1}, ..., \Delta_{4}$ are given in BB2010 Equations (5.77)--(5.81):

\begin{align}
    \Delta_0 &= K \left( \eta K - 2 \right) \\
    \Delta_1 &= -2 \left( \eta K - 1 \right) \left( K + \Delta_0 \right) \\
    \Delta_2 &= \frac{1}{2} \Delta_1 \left( -4 \eta K + \Delta_1 + 4 \right) - \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_0 \\
    \Delta_3 &= \frac{1}{3} \left[ -\Delta_1^{3} + 3 \left( \eta K - 1 \right) \Delta_1^{2} + 3 \Delta_2 \Delta_1 - 6 \left( \eta K - 1 \right) \left( -\eta K + \Delta_2 + 1 \right) - 3 \frac{ a^{2} }{ M^{2} } \left( \eta K - 1 \right)^{2} \Delta_1 \right] \\
    \Delta_4 &= \frac{1}{12} \left\{ 6 \frac{ a^{2} }{ M^{2} } \left( \Delta_1^{2} - 2 \Delta_2 \right) \left( \eta K - 1 \right)^{2} + 3 \Delta_1^{4} - 8 \left( \eta K - 1 \right) \Delta_1^{3} - 12 \Delta_2 \Delta_1^{2} + 12 \left[ 2 \left( \eta K - 1 \right) \Delta_2 + \Delta_3 \right] \Delta_1 + 12 \left( \frac{94}{3} - \frac{41}{32} \pi^2 \right) \left( \eta K - 1 \right)^{2} + 6 \left[ \Delta_2^{2} - 4 \Delta_3 \left( \eta K - 1 \right) \right] \right\}
\end{align}


(5.37)--(5.40) and (5.71)--(5.72),

\begin{align}
    \Delta_{t} &= r^{2} \Delta_{u}(u) \\
    \Delta_{u}(u) &= A(u) + \frac{ a^{2} }{ M^{2} } u^2 \\
    \Delta_{r} &= \Delta_{t} D^{-1}(u) \\
    \Lambda_{t} &= \varpi^{4} - a^{2} \Delta_{t} \sin^{2} \theta \\
    \tilde{\omega}_{\rm fd} &= 2 a M r + \omega_1^{\rm fd} \eta \frac{ a M^{3} }{ r } + \omega_{2}^{\rm fd} \eta \frac{ M a^{3} }{ r }
\end{align}

with $u = \frac{ M }{ r }$ and frame-dragging strength adjustable parameters $\omega_1^{\rm fd}$, $\omega_2^{\rm fd}$.  Finally, from BB2010 Equations (5.41) and (5.42)

\begin{align}
    A(u) &= 1 - 2 u + 2 \eta u^{3} + \eta \left( \frac{94}{3} - \frac{41}{32} \pi^{2} \right) u^{4} \\
    D^{-1}(u) &= 1 + 6 \eta u^{2} + 2 \left( 26 - 3 \eta \right) \eta u^{3}.
\end{align}

<a id='notes'></a>
# Notes \[Back to [top](#toc)\]
$$\label{notes}$$

1. First, just break down the Hamiltonian as in the paper.  Break up into blocks as appropriate.
2. Use python markdown to write string of input variables:

```python
# Step 0a: declare instrinsic system paramters (masses and spin components)
m1, m2, s1x, s1y, s1z, s2x, s2y, s2z = sp.symbols("m1 m2 s1x s1y s1z s2x s2y s2z",real=True)

```
3. Write terms to files in backwards order!  As in:

    ```python
    #First write is this:
    #%%writefile SEOBNR/Hamiltonian.txt

    ```
   and afterward

    ```python
    #%%writefile -a SEOBNR/Hamiltonian.txt

    ```
   The goal is to split up the terms so that each $\LaTeX$-expression at the bottom of a markdown cell can be easily compared to the "bigstring" expression that is SymPy compatible in a code cell directly below the markdown cell.

<a id='latex_pdf_output'></a>

# Output: Output this module to $\LaTeX$-formatted PDF file \[Back to [top](#toc)\]
$$\label{latex_pdf_output}$$

In [2]:
!jupyter nbconvert --to latex --template latex_nrpy_style.tplx Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!pdflatex -interaction=batchmode Tutorial-SEOBNR_Documentation.ipynb
!rm -f Tut*.out Tut*.aux Tut*.log

[NbConvertApp] Converting notebook Tutorial-SEOBNR_Documentation.ipynb to latex
[NbConvertApp] Writing 41167 bytes to Tutorial-SEOBNR_Documentation.tex
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
This is pdfTeX, Version 3.14159265-2.6-1.40.18 (TeX Live 2017/Debian) (preloaded format=pdflatex)
 restricted \write18 enabled.
entering extended mode
